In [15]:
!gdown --id 1KmoZYmPzxrrtR4a3mSfPIumbDwf_Mrpw > /dev/null 2>&1
!gdown --id 19vK08esg9hQtD8JB-9jm7Qkck7nZ-ZqR > /dev/null 2>&1

In [16]:
!pip install transformers > /dev/null 2>&1
!pip install --upgrade accelerate > /dev/null 2>&1

In [17]:
import torch
import torch.nn.functional as F
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
from collections import Counter
import re

In [18]:
%%time
df = pd.read_feather(r'comments_cleaned_feather.feather')

CPU times: user 1.77 s, sys: 1.54 s, total: 3.32 s
Wall time: 4.83 s


In [19]:
with open('slur_list.txt', 'r') as file:
    slur_words = [word.strip() for word in file.readlines()]

In [20]:
header = np.array(df.columns)
df = df[~np.all(df.values == header, axis=1)]
df = df[~df['videoId'].str.contains(r'\bvideoId\b', case=False, regex=True)]
pattern = r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z$'
df = df[df['publishedAt'].str.contains(pattern)]
df['textOriginal'] = df['textOriginal'].astype(str)

<ipython-input-20-983c3e9dee07>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['textOriginal'] = df['textOriginal'].astype(str)


In [21]:
model_path = "ganeshkharad/gk-hinglish-sentiment"

In [22]:
# https://huggingface.co/ganeshkharad/gk-hinglish-sentiment
tokenizerg = BertTokenizer.from_pretrained(model_path)
modelg = BertForSequenceClassification.from_pretrained(model_path)

In [23]:
labels = ["Negative", "Neutral", "Positive"]

In [24]:
def predict_sentiment(text):
    encoded_input = tokenizerg(text, return_tensors='pt',truncation=True, max_length=512)
    output = modelg(**encoded_input)
    probabilities = F.softmax(output.logits, dim=1)
    predicted_label_index = torch.argmax(probabilities, dim=1).item()
    predicted_label = labels[predicted_label_index]
    predicted_score = probabilities[0][predicted_label_index].item()
    # label_scores = {labels[i]: probabilities[0][i].item() for i in range(len(labels))}
    return predicted_label, predicted_score

In [25]:
text = "abe tu jaa re"
# text = "😂😂😂"
label, score = predict_sentiment(text)

In [26]:
print(f"{label}, {score}")

Neutral, 0.9996445178985596


In [29]:
testDf = df.head(100)

In [30]:
%%time
testDf['label'], testDf['score'] = zip(*testDf['textOriginal'].apply(predict_sentiment))

CPU times: user 24.7 s, sys: 47.9 ms, total: 24.8 s
Wall time: 31.8 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
